# perplexity 评估

衡量模型对测试数据的预测能力，数值越低表示模型性能越好。


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import torch
import math

# 加载预训练模型和 tokenizer
model_name = "C:/Users/Admin/Desktop/model/qwen/Qwen2___5-0___5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 加载评估数据集
data_files = {"test": "C:/Users/Admin/Desktop/Data/huatuo_train.json"}
dataset = load_dataset("json", data_files=data_files, split="test")


# 定义计算困惑度的函数
def compute_perplexity(model, tokenizer, dataset):
    model.eval()
    total_loss = 0.0
    total_length = 0

    for example in dataset:
        for conversation in example["conversations"]:
            if conversation["from"] == "human":
                inputs = tokenizer(
                    conversation["value"],
                    return_tensors="pt",
                    truncation=True,
                    padding=True,
                )
                input_ids = inputs.input_ids.to(model.device)
                with torch.no_grad():
                    outputs = model(input_ids, labels=input_ids)
                    loss = outputs.loss
                    total_loss += loss.item() * input_ids.size(1)
                    total_length += input_ids.size(1)

    perplexity = math.exp(total_loss / total_length)
    return perplexity


# 计算困惑度
perplexity = compute_perplexity(model, tokenizer, dataset)
print(f"困惑度: {perplexity}")

困惑度: 89.75600468401245
